In [1]:
import pickle
dataset,countdic=pickle.load(open('./data_and_vocab.pkl','rb'))

# データ加工部

In [6]:
dataset=sorted(dataset,key=lambda x:x[0])

In [252]:
index=0
indexlist=[]
for i in range(len(dataset)):
    if i==0:
        indexlist.append(index)
    if (dataset[i][0]-dataset[i-1][0]).total_seconds()>180:
        index+=1
    indexlist.append(index)

In [253]:
len(indexlist),max(indexlist)

(64204, 29813)

In [254]:
merged_dataset=[[None,[]] for i in range(max(indexlist)+1)]
for data,i in zip(dataset,indexlist):
    if merged_dataset[i][0] is None:
        merged_dataset[i][0]=data[0]
    merged_dataset[i][1]+=[j for j in zip(data[1].words,data[1].postags)]

# 辞書づくり部

In [255]:
vocabs=[(j,i) for i in countdic for j in countdic[i] if countdic[i][j]>20]

In [256]:
vocab2index={i[1]:i[0] for i in enumerate(vocabs)}

In [257]:
tmps=[]
feature=[]
for i in merged_dataset:
    tmp=[j for j in i[1] if j in vocabs]
    tmps.append(tmp)
    feature.append([[vocab2index[j],tmp.count(j)] for j in set(tmp)])

In [258]:
stop_words=['0','1','2','3','4','5','6','7','8','9','ない','する','いう','よう','',
           '僕','そう','こと','それ','さん','思っ','感じ','Minathuki','まぁ','ある','なる','これ','いる','なく','',
           'いい','あれ','うち','なん','ちゃん','RT','どっ','どう','思う','ある','なっ','くる','やっ','あっ','なり',
            '本日','今日','明日','みんな','自分','もの','とこ','やつ','capp','データ','日本','おっ','TL',
           'ツイ', 'リプ','はず','ほう','ひと','ため','あと','わけ','もん','最近','必要','とき']

In [259]:
input_data=[[j[0] for j in i if j[0] not in stop_words and len(j[0])>1 and j[1]=='名詞'] for i in tmps]

In [260]:
import gensim
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary(input_data)
#dictionary.filter_extremes(no_below=30, no_above=0.1)

In [261]:
corpus = []
for word in input_data:
    bow = dictionary.doc2bow(word)
    corpus.append(bow)


In [262]:
topic_N = 4

lda = gensim.models.ldamodel.LdaModel(
          corpus     = corpus
        , num_topics = topic_N
        , id2word    = dictionary
      )

for i in range(topic_N):
    print("\n")
    print("="*80)
    print("TOPIC {0}\n".format(i))
    topic = lda.show_topic(i)
    for t in topic:
        print("{0:20s}{1}".format(t[0], t[1]))



TOPIC 0

時間                  0.024832112714648247
人生                  0.02263304404914379
アニメ                 0.020667990669608116
先生                  0.019204696640372276
フレンズ                0.013757986016571522
サナララ                0.013366804458200932
のぞみ                 0.013270673342049122
本当                  0.012527930550277233
ところ                 0.012380399741232395
最高                  0.012006085366010666


TOPIC 1

仕事                  0.026127759367227554
気持ち                 0.020275553688406944
社会                  0.017976339906454086
だめ                  0.012662854045629501
概念                  0.012033949606120586
おじ                  0.008770833723247051
無理                  0.008557888679206371
部屋                  0.008020891807973385
PC                  0.007842323742806911
問題                  0.007713694125413895


TOPIC 2

魔法                  0.02340560220181942
少女                  0.022581906989216805
先生                  0.013330072164535522
労働                  0.0101

In [243]:
lda.get_term_topics('サナララ')

[(2, 0.018929616)]